# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1100c17400>
├── 'a' --> tensor([[-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358]])
└── 'x' --> <FastTreeValue 0x7f1100c17760>
    └── 'c' --> tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                        [ 1.5039, -0.8793,  0.3076, -0.6039],
                        [-1.3072, -0.2627,  0.5228, -0.7813]])

In [4]:
t.a

tensor([[-0.1379,  0.0042,  0.1328],
        [-0.5109,  0.0227, -0.7358]])

In [5]:
%timeit t.a

59.1 ns ± 0.0532 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1100c17400>
├── 'a' --> tensor([[-0.4503,  0.6251,  0.3961],
│                   [-0.5962, -0.2087,  1.3191]])
└── 'x' --> <FastTreeValue 0x7f1100c17760>
    └── 'c' --> tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                        [ 1.5039, -0.8793,  0.3076, -0.6039],
                        [-1.3072, -0.2627,  0.5228, -0.7813]])

In [7]:
%timeit t.a = new_value

64.2 ns ± 0.0453 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1379,  0.0042,  0.1328],
               [-0.5109,  0.0227, -0.7358]]),
    x: Batch(
           c: tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813]]),
       ),
)

In [10]:
b.a

tensor([[-0.1379,  0.0042,  0.1328],
        [-0.5109,  0.0227, -0.7358]])

In [11]:
%timeit b.a

57.5 ns ± 0.0266 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.8445, -1.0724,  1.2476],
               [ 0.8036, -0.7160,  0.3466]]),
    x: Batch(
           c: tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.117 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

866 ns ± 0.436 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 19.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

133 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 200 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1100c12250>
├── 'a' --> tensor([[[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]],
│           
│                   [[-0.1379,  0.0042,  0.1328],
│                    [-0.5109,  0.0227, -0.7358]]])
└── 'x' --> <FastTreeValue 0x7f1100c122e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 24.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f10568c0fa0>
├── 'a' --> tensor([[-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358],
│                   [-0.1379,  0.0042,  0.1328],
│                   [-0.5109,  0.0227, -0.7358]])
└── 'x' --> <FastTreeValue 0x7f1100c8c970>
    └── 'c' --> tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                        [ 1.5039, -0.8793,  0.3076, -0.6039],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 29 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.9 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.0258, -0.6893, -0.3963,  1.9491],
                       [ 1.5039, -0.8793,  0.3076, -0.6039],
                       [-1.3072, -0.2627,  0.5228, -0.7813]],
              
                      [[-0.0258, -0.6893, -0.3963,  1.9491],
                       [ 1.5039, -0.8793,  0.3076, -0.6039],
                       [-1.3072, -0.2627,  0.5228, -0.7813]],
              
                      [[-0.0258, -0.6893, -0.3963,  1.9491],
                       [ 1.5039, -0.8793,  0.3076, -0.6039],
                       [-1.3072, -0.2627,  0.5228, -0.7813]],
              
                      [[-0.0258, -0.6893, -0.3963,  1.9491],
                       [ 1.5039, -0.8793,  0.3076, -0.6039],
                       [-1.3072, -0.2627,  0.5228, -0.7813]],
              
                      [[-0.0258, -0.6893, -0.3963,  1.9491],
                       [ 1.5039, -0.8793,  0.3076, -0.6039],
                       [-1.3072, -0.2627,  0.5228, -0.7813]],

In [26]:
%timeit Batch.stack(batches)

77 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813],
                      [-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813],
                      [-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813],
                      [-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813],
                      [-0.0258, -0.6893, -0.3963,  1.9491],
                      [ 1.5039, -0.8793,  0.3076, -0.6039],
                      [-1.3072, -0.2627,  0.5228, -0.7813],
                      [-0.0258, -0.6893, -0.3963,  1.9491],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 119 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

303 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
